In [97]:
import re
import pandas as pd
from io import StringIO
import nltk
import numpy as np
import json

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

lemma = nltk.wordnet.WordNetLemmatizer()

ps = PorterStemmer()

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/matt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [98]:
def extractDataFrame(fileName):
  with open(fileName) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
    df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")
  return df

In [99]:
def splitLabels(labels):
  return [int(label) for label in labels.split(' ')]

In [100]:
def tokenise(caption):
    # Replace non-alphabetic characters with single whitespace
    caption = re.sub(r'[^a-zA-Z\s]', ' ', caption.lower())
    # Remove any whitespace that appears in sequence
    caption = re.sub(r"\s+", " ", caption)
    # Remove new leading and trailing whitespace
    caption = caption.strip()
    # Apply stemmer to each remaining word in sentence
    caption = lemma.lemmatize(caption)
    caption = ps.stem(caption)

    # Tokenize
    word_tokens = word_tokenize(caption)
    # Remove stop words
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    # Replace value with tokenised data
    return filtered_sentence

In [101]:
train = extractDataFrame('data/train.csv')

train['Tokens'] = train['Caption'].apply(tokenise)
train['Labels'] = train['Labels'].apply(splitLabels)

uniqueLabels = set()
for labels in train['Labels']:
  uniqueLabels.update(labels)

numUniqueLabels = len(uniqueLabels)
maxLabel = max(uniqueLabels)

print(uniqueLabels)
print('Unique labels:', numUniqueLabels)
print('Max label value:', maxLabel)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19}
Unique labels: 18
Max label value: 19


In [102]:
def labelMapping(uLabels):
  labelToIdMapping = {}
  idToLabelMapping = {}
  for idx, label in enumerate(uLabels):
    labelToIdMapping[label] = idx
    idToLabelMapping[idx] = label
  return labelToIdMapping, idToLabelMapping

In [103]:
labelToIdMapping, idToLabelMapping = labelMapping(uniqueLabels)

print(idToLabelMapping)

def mapLabelToId(labels):
  ids = []
  for label in labels:
    ids.append(labelToIdMapping[label])
  return ids

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 13, 12: 14, 13: 15, 14: 16, 15: 17, 16: 18, 17: 19}


In [104]:
def oneHotEncodeLabel(labels):
  oneHotClasses = np.zeros([numUniqueLabels])
  for label in labels:
    # Onehot encode each label in the list
    oneHotClasses[label] = 1.
  return oneHotClasses.tolist()

In [105]:
train['LabelIds'] = train['Labels'].apply(mapLabelToId)
train['EncodedLabels'] = train['LabelIds'].apply(oneHotEncodeLabel)

print(train['Labels'][3])
print(train['LabelIds'][3])
print(train['EncodedLabels'][3])

[8, 3, 13]
[7, 2, 11]
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [106]:
tokenLengths = []
for x in range(train.shape[0]):
    tokenLengths.append(len(train["Tokens"][x]))

maxTokenLength = max(tokenLengths)
print('Max token Length:', maxTokenLength)

Max token Length: 28


In [107]:
# Initialise vocabulary including padding and unknown word tokens
# Is end of sentence needed? '[EOS]'
vocabulary = set(['[PAD]', '[UNK]'])

for tokens in train["Tokens"]:
  vocabulary.update(tokens)

vocabulary = sorted(vocabulary)
print(vocabulary)
print('Unique words:', len(vocabulary))

['[PAD]', '[UNK]', 'aaron', 'abandon', 'abandoned', 'abat', 'abed', 'able', 'aboard', 'abov', 'absolutley', 'abstract', 'acc', 'accents', 'accepting', 'accessori', 'accessories', 'accolades', 'accompanied', 'across', 'acrossed', 'act', 'acting', 'action', 'actively', 'actor', 'actors', 'actress', 'acurly', 'ad', 'adamantly', 'add', 'added', 'adjacent', 'adjoining', 'adjustable', 'adjusted', 'adjusting', 'adjusts', 'admire', 'admiring', 'adopted', 'adorable', 'adoring', 'adorned', 'ads', 'adult', 'adults', 'advanced', 'advantage', 'adventure', 'advertis', 'advertisement', 'advertisements', 'advertises', 'advertising', 'advertsment', 'advice', 'advising', 'aerial', 'afar', 'affair', 'affect', 'affectionate', 'affixed', 'afloat', 'african', 'afro', 'afternoon', 'age', 'aged', 'agent', 'ages', 'aggressively', 'ago', 'agricultural', 'ahead', 'aid', 'aimed', 'aiming', 'aims', 'air', 'airborne', 'aircraft', 'airfield', 'airforce', 'airfrance', 'airiel', 'airlin', 'airline', 'airliner', 'airli

In [108]:
dictionary = {}
idToDictionary = {}
for idx, word in enumerate(vocabulary):
  dictionary[word] = idx
  idToDictionary[idx] = word

with open('./processed-data/dictionary.json', 'w', encoding='utf-8') as f:
    json.dump(dictionary, f, ensure_ascii=False, indent=4)

In [109]:
# Add dictionary IDs column to dataframe
def addDictionaryIds(tokens):
    ids = []
    for token in tokens:
      if token in dictionary:
        ids.append(dictionary[token])
      else:
        ids.append(dictionary['[UNK]'])
    return ids

In [110]:
train['DictionaryIds'] = train['Tokens'].apply(addDictionaryIds)

In [111]:
def addTokenPadding(tokens):
  padding = maxTokenLength - len(tokens)
  return tokens + padding*[dictionary['[PAD]']]

In [112]:
train['TokensWithPadding'] = train['DictionaryIds'].apply(addTokenPadding)
train.head()

,ImageID,Labels,Caption,Tokens,LabelIds,EncodedLabels,DictionaryIds,TokensWithPadding
0,0.jpg,[1],Woman in swim suit holding parasol on sunny day.,"[woman, swim, suit, holding, parasol, sunny, day]",[0],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7236, 6376, 6297, 3077, 4457, 6316, 1714]","[7236, 6376, 6297, 3077, 4457, 6316, 1714, 0, ..."
1,1.jpg,"[1, 19]",A couple of men riding horses on top of a gree...,"[couple, men, riding, horses, top, green, field]","[0, 17]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1524, 3891, 5237, 3117, 6650, 2827, 2351]","[1524, 3891, 5237, 3117, 6650, 2827, 2351, 0, ..."
2,2.jpg,[1],They are brave for riding in the jungle on tho...,"[brave, riding, jungle, eleph]",[0],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[754, 5237, 3368, 2093]","[754, 5237, 3368, 2093, 0, 0, 0, 0, 0, 0, 0, 0..."
3,3.jpg,"[8, 3, 13]",a black and silver clock tower at an intersect...,"[black, silver, clock, tower, intersection, ne...","[7, 2, 11]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[616, 5718, 1257, 6684, 3267, 4138, 6738]","[616, 5718, 1257, 6684, 3267, 4138, 6738, 0, 0..."
4,4.jpg,"[8, 3, 7]",A train coming to a stop on the tracks out side.,"[train, coming, stop, tracks, sid]","[7, 2, 6]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[6704, 1367, 6178, 6694, 5677]","[6704, 1367, 6178, 6694, 5677, 0, 0, 0, 0, 0, ..."


In [113]:
# Setup test data

# Get test CSV dataframe
test = extractDataFrame('data/test.csv')

# Tokenise
test['Tokens'] = test['Caption'].apply(tokenise)
# Apply dictionary IDs
test['DictionaryIds'] = test['Tokens'].apply(addDictionaryIds)
# Add padding
test['TokensWithPadding'] = test['DictionaryIds'].apply(addTokenPadding)

test.head()

,ImageID,Caption,Tokens,DictionaryIds,TokensWithPadding
0,30000.jpg,A little girl waring a krispy kreme hat holdin...,"[little, girl, waring, krispy, kreme, hat, hol...","[3654, 2720, 7076, 3448, 3446, 2966, 3077, 731...","[3654, 2720, 7076, 3448, 3446, 2966, 3077, 731..."
1,30001.jpg,A beautiful young woman holding an orange fris...,"[beautiful, young, woman, holding, orange, fri...","[508, 7310, 7236, 3077, 4304, 2598]","[508, 7310, 7236, 3077, 4304, 2598, 0, 0, 0, 0..."
2,30002.jpg,A group of people sitting on couch next to a c...,"[group, people, sitting, couch, next, coffee]","[2863, 4586, 5739, 1511, 4176, 1319]","[2863, 4586, 5739, 1511, 4176, 1319, 0, 0, 0, ..."
3,30003.jpg,A person on a snowboard rides on the hill.,"[person, snowboard, rides, hil]","[4609, 5900, 5235, 3047]","[4609, 5900, 5235, 3047, 0, 0, 0, 0, 0, 0, 0, ..."
4,30004.jpg,A man riding a skateboard with a helmet on in ...,"[man, riding, skateboard, helmet, con]","[3778, 5237, 5749, 3017, 1400]","[3778, 5237, 5749, 3017, 1400, 0, 0, 0, 0, 0, ..."


In [114]:
train.to_csv('./processed-data/train.csv')
test.to_csv('./processed-data/test.csv')